# CancerRxGene
This dataset has two different forms - processed and raw. My initial thoughts are to use the processed dataset, as it has some basic quality control measures applied, and gives an IC50 value. The raw data however contains no IC50 value. I will quickly do a sense check on this however.

In [14]:
import requests
import time

import pandas as pd

In [15]:
processed_df = pd.read_excel("/Users/sethhowes/Desktop/FS-Tox/data/raw/cancerrx.xlsx")
raw_df = pd.read_csv("/Users/sethhowes/Desktop/FS-Tox/data/raw/cancerrx.csv")

In [16]:
processed_df.head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038


In [17]:
processed_df.shape

(242036, 19)

In [18]:
raw_df.head()

,RESEARCH_PROJECT,BARCODE,SCAN_ID,DATE_CREATED,SCAN_DATE,CELL_ID,MASTER_CELL_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,SEEDING_DENSITY,DRUGSET_ID,ASSAY,DURATION,POSITION,TAG,DRUG_ID,CONC,INTENSITY
0,GDSC_SA,3230,2945,2015-02-13T00:00:00Z,2015-02-17T00:00:00Z,4712,198,753608,PC-14,SIDM00237,250,159,Glo,4,1,UN-USED,NaN,NaN,0
1,GDSC_SA,3230,2945,2015-02-13T00:00:00Z,2015-02-17T00:00:00Z,4712,198,753608,PC-14,SIDM00237,250,159,Glo,4,2,UN-USED,NaN,NaN,0
2,GDSC_SA,3230,2945,2015-02-13T00:00:00Z,2015-02-17T00:00:00Z,4712,198,753608,PC-14,SIDM00237,250,159,Glo,4,3,UN-USED,NaN,NaN,398
3,GDSC_SA,3230,2945,2015-02-13T00:00:00Z,2015-02-17T00:00:00Z,4712,198,753608,PC-14,SIDM00237,250,159,Glo,4,4,UN-USED,NaN,NaN,796
4,GDSC_SA,3230,2945,2015-02-13T00:00:00Z,2015-02-17T00:00:00Z,4712,198,753608,PC-14,SIDM00237,250,159,Glo,4,5,UN-USED,NaN,NaN,796


In [19]:
raw_df.shape

(15069449, 19)

The raw dataframe clearly has a lot of information that is redundant for our purposes. This includes individual well readings.

In [13]:
del raw_df

I now want to quickly check missingness.

In [21]:
processed_df.isna().sum()

DATASET                0
NLME_RESULT_ID         0
NLME_CURVE_ID          0
COSMIC_ID              0
CELL_LINE_NAME         0
SANGER_MODEL_ID        0
TCGA_DESC           1067
DRUG_ID                0
DRUG_NAME              0
PUTATIVE_TARGET    27155
PATHWAY_NAME           0
COMPANY_ID             0
WEBRELEASE             0
MIN_CONC               0
MAX_CONC               0
LN_IC50                0
AUC                    0
RMSE                   0
Z_SCORE                0
dtype: int64

This missing data shouldn't really matter. They won't be important in defining an assay.

## Defining assay

In [24]:
processed_df.head()

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.0001,0.1,-3.741620,0.733992,0.128066,-0.807038


It appears that the following columns could be used for the assay creation:
- `NLME_RESULT_ID` - Identifier for the fitted dose response. Doesn't appear to be useful
- `COSMIC_ID` - cell identifier for the COSMIC database
- `CELL_LINE_NAME` - name of the cell line
- `SANGER_MODEL_ID` - the id of the cell model on the Sanger catalogue

I wonder what degree of overlap there is between the latter three cellular identifiers.

In [30]:
assay_components = ["CELL_LINE_NAME", "COSMIC_ID", "SANGER_MODEL_ID"]
processed_df.loc[:, assay_components].value_counts()

CELL_LINE_NAME  COSMIC_ID  SANGER_MODEL_ID
PC-14           753608     SIDM00237          295
SW620           905962     SIDM00841          295
HT-29           905939     SIDM00136          295
A375            906793     SIDM00795          294
MHH-ES-1        908134     SIDM00386          294
                                             ... 
SNU-1040        1659823    SIDM00217          148
COR-L95         1297439    SIDM00521           99
LNZTA3WT4       1240170    SIDM01220           99
HCC202          1290906    SIDM00870           14
RH-18           971774     SIDM00454           12
Name: count, Length: 969, dtype: int64

In [39]:
processed_df.groupby(assay_components).size().reset_index(name="count").sort_values(by="count", ascending=False)

,CELL_LINE_NAME,COSMIC_ID,SANGER_MODEL_ID,count
726,PC-14,753608,SIDM00237,295
883,SW620,905962,SIDM00841,295
303,HT-29,905939,SIDM00136,295
20,A375,906793,SIDM00795,294
491,MHH-ES-1,908134,SIDM00386,294
...,...,...,...,...
839,SNU-1040,1659823,SIDM00217,148
129,COR-L95,1297439,SIDM00521,99
444,LNZTA3WT4,1240170,SIDM01220,99
268,HCC202,1290906,SIDM00870,14


We can see that these different approaches are identical, indicating that there is a one-to-one mapping between these three identifiers. I can therefore group by any of them and the resultant assay would be the same.

In [42]:
processed_df.groupby("CELL_LINE_NAME").filter(lambda x: len(x) >= 24)

,DATASET,NLME_RESULT_ID,NLME_CURVE_ID,COSMIC_ID,CELL_LINE_NAME,SANGER_MODEL_ID,TCGA_DESC,DRUG_ID,DRUG_NAME,PUTATIVE_TARGET,PATHWAY_NAME,COMPANY_ID,WEBRELEASE,MIN_CONC,MAX_CONC,LN_IC50,AUC,RMSE,Z_SCORE
0,GDSC2,401,18945558,683667,PFSK-1,SIDM01132,MB,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-1.462148,0.930105,0.088999,0.432482
1,GDSC2,401,18945796,684052,A673,SIDM00848,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-4.869447,0.614932,0.111423,-1.420322
2,GDSC2,401,18946078,684057,ES5,SIDM00263,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.360684,0.790953,0.142754,-0.599894
3,GDSC2,401,18946335,684059,ES7,SIDM00269,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-5.045014,0.592624,0.135642,-1.515791
4,GDSC2,401,18946617,684062,EW-11,SIDM00203,UNCLASSIFIED,1003,Camptothecin,TOP1,DNA replication,1046,Y,0.000100,0.1,-3.741620,0.733992,0.128066,-0.807038
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242031,GDSC2,401,19187490,1659928,SNU-175,SIDM00216,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.134495,0.976798,0.074441,0.159946
242032,GDSC2,401,19187943,1660034,SNU-407,SIDM00214,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,8.575555,0.913182,0.057743,-1.626059
242033,GDSC2,401,19188201,1660035,SNU-61,SIDM00194,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.520666,0.974889,0.058094,0.602364
242034,GDSC2,401,19188741,1674021,SNU-C5,SIDM00498,COREAD,2499,N-acetyl cysteine,Metabolism,Metabolism,1101,Y,2.001054,2000.0,10.701430,0.970009,0.100980,0.809457


## Getting SMILES - PubChem API
I now need to convert the `DRUG_NAME` to SMILES notation.

In [48]:
def get_smiles(drug_name):
    base_url = "https://pubchem.ncbi.nlm.nih.gov/rest/pug"
    url = f"{base_url}/compound/name/{drug_name}/property/IsomericSMILES/JSON"
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # If the response was successful, no Exception will be raised
    except requests.HTTPError as http_err:
        print(f'HTTP error occurred for {drug_name}: {http_err}')
        return None
    except Exception as err:
        print(f'An error occurred for {drug_name}: {err}')
        return None

    return response.json()['PropertyTable']['Properties'][0]['IsomericSMILES']

drug_names = processed_df["DRUG_NAME"].unique()

smiles_dict = {}
for drug_name in drug_names:
    smiles = get_smiles(drug_name)
    if smiles is not None:
        smiles_dict[drug_name] = smiles
    time.sleep(0.2)  # wait for 0.2 before sending another request

HTTP error occurred for Nutlin-3a (-): 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Nutlin-3a%20(-)/property/IsomericSMILES/JSON
HTTP error occurred for Bleomycin (50 uM): 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/Bleomycin%20(50%20uM)/property/IsomericSMILES/JSON
HTTP error occurred for IAP_5620: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/IAP_5620/property/IsomericSMILES/JSON
HTTP error occurred for BDOCA000347a: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/BDOCA000347a/property/IsomericSMILES/JSON
HTTP error occurred for BDF00022089a: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/BDF00022089a/property/IsomericSMILES/JSON
HTTP error occurred for BDILV000379a: 404 Client Error: PUGREST.NotFound for url: https://

HTTP error occurred for N30652-18-1: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/N30652-18-1/property/IsomericSMILES/JSON
HTTP error occurred for N29087-69-1: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/N29087-69-1/property/IsomericSMILES/JSON
HTTP error occurred for HKMTI-1-005: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/HKMTI-1-005/property/IsomericSMILES/JSON
HTTP error occurred for VTP-A: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/VTP-A/property/IsomericSMILES/JSON
HTTP error occurred for VTP-B: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/VTP-B/property/IsomericSMILES/JSON
HTTP error occurred for PBD-288: 404 Client Error: PUGREST.NotFound for url: https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/P

In [51]:
# Add the smiles values as a new column
processed_df['SMILES'] = processed_df['DRUG_NAME'].map(smiles_dict)

# Remove rows where 'SMILES' is NaN
processed_df = processed_df.dropna(subset=['SMILES'])

In [52]:
processed_df.shape

(198784, 20)

In [53]:
processed_df = processed_df.groupby("CELL_LINE_NAME").filter(lambda x: len(x) >= 24)

In [54]:
processed_df.shape

(198760, 20)

In [55]:
processed_df.to_csv("/Users/sethhowes/Desktop/FS-Tox/data/raw/cancerrx_pubchem.csv")